In [128]:

from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random

from icecream import ic
from dataclasses import dataclass
from tqdm import tqdm
import logging

In [129]:
countries = ['Russia', 'Italy', 'Vanuatu', 'China', 'US']

def init(country):
    global CITIES, DIST_MATRIX
    # Load cities data
    CITIES = pd.read_csv(f'{country}.csv'.lower(), header=None, names=['name', 'lat', 'lon'])

    # Create distance matrix
    DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
    for c1, c2 in combinations(CITIES.itertuples(), 2):
        DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
            (c1.lat, c1.lon), (c2.lat, c2.lon)
        ).km
    

def compute_tour_distance(path, distance_matrix):
    total_distance = 0.0
    for i in range(len(path) - 1):
        #city_from = path[i]
        #city_to = path[i + 1]
        total_distance += distance_matrix[path[i]][path[i + 1]]
    return total_distance     

In [130]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## First Greedy Algorithm: nearest neighbor

In [131]:

for country in countries:

    init(country)
    print(country)

    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    city = 0
    visited[city] = True
    tsp = list()
    tsp.append(int(city))
    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        print(f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)")
        visited[closest] = True
        city = closest
        tsp.append(int(city))
    print(f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)")
    tsp.append(tsp[0])


    logging.info(f"{country} result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

INFO:root:Russia result: Found a path of 167 steps, total length 42334.16km
INFO:root:Italy result: Found a path of 46 steps, total length 4436.03km
INFO:root:Vanuatu result: Found a path of 8 steps, total length 1475.53km


Russia
step: Abakan -> Krasnoyarsk (276.58km)
step: Krasnoyarsk -> Achinsk (161.71km)
step: Achinsk -> Kemerovo (296.59km)
step: Kemerovo -> Leninsk‐Kuznetskiy (74.76km)
step: Leninsk‐Kuznetskiy -> Prokopyevsk (91.87km)
step: Prokopyevsk -> Novokuznetsk (30.63km)
step: Novokuznetsk -> Biysk (187.38km)
step: Biysk -> Barnaul (132.82km)
step: Barnaul -> Novosibirsk (194.50km)
step: Novosibirsk -> Tomsk (206.90km)
step: Tomsk -> Seversk (14.97km)
step: Seversk -> Rubtsovsk (613.13km)
step: Rubtsovsk -> Omsk (647.47km)
step: Omsk -> Tobolsk (475.40km)
step: Tobolsk -> Tyumen (200.98km)
step: Tyumen -> Kurgan (189.69km)
step: Kurgan -> Kopeysk (236.87km)
step: Kopeysk -> Chelyabinsk (14.72km)
step: Chelyabinsk -> Miass (87.20km)
step: Miass -> Zlatoust (33.88km)
step: Zlatoust -> Pervouralsk (194.64km)
step: Pervouralsk -> Yekaterinburg (40.19km)
step: Yekaterinburg -> Kamensk‐Uralskiy (95.20km)
step: Kamensk‐Uralskiy -> Nizhniy Tagil (205.75km)
step: Nizhniy Tagil -> Perm (220.37km)
step: 

INFO:root:China result: Found a path of 726 steps, total length 63962.92km


China
step: Acheng -> Harbin (33.60km)
step: Harbin -> Shuangcheng (53.02km)
step: Shuangcheng -> Yushu (61.85km)
step: Yushu -> Wuchang (47.68km)
step: Wuchang -> Shulan (59.07km)
step: Shulan -> Jishu (17.91km)
step: Jishu -> Jilin city (50.81km)
step: Jilin city -> Jiutai (65.06km)
step: Jiutai -> Dehui (43.68km)
step: Dehui -> Changchun (78.49km)
step: Changchun -> Gongzhuling (59.12km)
step: Gongzhuling -> Siping (54.24km)
step: Siping -> Liaoyuan (71.76km)
step: Liaoyuan -> Meihekou (60.38km)
step: Meihekou -> Panshi (55.16km)
step: Panshi -> Huadian (56.40km)
step: Huadian -> Jiaohe (96.49km)
step: Jiaohe -> Dunhua (82.15km)
step: Dunhua -> Helong (110.22km)
step: Helong -> Longjing (42.88km)
step: Longjing -> Yanji (14.70km)
step: Yanji -> Tumen (26.45km)
step: Tumen -> Huichun (46.09km)
step: Huichun -> Ningan (179.31km)
step: Ningan -> Hailin (26.54km)
step: Hailin -> Mudanjiang (18.30km)
step: Mudanjiang -> Muleng (81.67km)
step: Muleng -> Hengshan (43.08km)
step: Hengshan -

INFO:root:US result: Found a path of 326 steps, total length 48050.03km


US
step: Abilene -> Wichita Falls (196.78km)
step: Wichita Falls -> Denton (149.68km)
step: Denton -> Lewisville (23.92km)
step: Lewisville -> Carrollton (10.02km)
step: Carrollton -> Plano (15.78km)
step: Plano -> Allen (9.27km)
step: Allen -> McKinney (10.45km)
step: McKinney -> Frisco (15.17km)
step: Frisco -> Richardson (22.38km)
step: Richardson -> Garland (10.04km)
step: Garland -> Mesquite (16.57km)
step: Mesquite -> Dallas (16.56km)
step: Dallas -> Irving (20.41km)
step: Irving -> Grand Prairie (19.83km)
step: Grand Prairie -> Fort Worth (32.28km)
step: Fort Worth -> Waco (136.07km)
step: Waco -> Killeen (74.58km)
step: Killeen -> Round Rock (61.72km)
step: Round Rock -> Austin (25.47km)
step: Austin -> San Antonio (118.67km)
step: San Antonio -> Corpus Christi (220.93km)
step: Corpus Christi -> Edinburg (175.38km)
step: Edinburg -> McAllen (13.47km)
step: McAllen -> Brownsville (82.29km)
step: Brownsville -> Laredo (263.79km)
step: Laredo -> Sugar Land (440.43km)
step: Sugar L

## Evolutionary algorithm

In [132]:

#definisci class individual
@dataclass
class Individual:
    genome: list
    fitness: float = None

    def compute_fitness(self):
        total_distance = 0
        for i in range(len(self.genome) - 1):
            total_distance += DIST_MATRIX[self.genome[i], self.genome[i + 1]]
        total_distance += DIST_MATRIX[self.genome[-1], self.genome[0]]  #torno alla prima città
        self.fitness = 1/total_distance 



## Parent sel

In [133]:
def parent_sel_uniform(population):
    return random.choice(population)

def parent_sel_tournament(population, tournament_size=3):
    candidates = random.sample(population, tournament_size)
    return max(candidates, key=lambda ind: ind.fitness)

In [134]:
def xover(p1, p2):
    size = len(p1.genome)
    start, end = sorted(random.sample(range(1, size), 2))
    child_genome = [None] * size
    child_genome[start:end] = p1.genome[start:end]
    
    current_index = 0
    for gene in p2.genome:
        if gene not in child_genome:
            while child_genome[current_index] is not None:
                current_index += 1
            child_genome[current_index] = gene
            
    return Individual(genome=child_genome)
            

def mutation(individual, ):   #inversion mut
    if np.random.rand() < 0.1:
        idx1, idx2 = random.sample(range(1, len(individual.genome)), 2) 
        if idx1 > idx2:
            idx1, idx2 = idx2, idx1
        individual.genome[idx1:idx2+1] = reversed(individual.genome[idx1:idx2+1])
    return individual          

## Algortimo

### uniform selection

In [135]:
POPULATION_SIZE = 100
MAX_GENERATIONS = 1000
def genetic_alg( country='Italy'):
    
    population = []
    for _ in range(POPULATION_SIZE):
        unique_cities = random.sample(range(1, len(CITIES)), len(CITIES) - 1)
        genome = [0] + unique_cities 
        population.append(Individual(genome=genome))

    for ind in population:
        ind.compute_fitness()

    for _ in tqdm(range(MAX_GENERATIONS)):
        new_population = []

        while len(new_population) < POPULATION_SIZE:

            #uniform sel
            parent1 = parent_sel_uniform(population)
            parent2 = parent_sel_uniform(population)
            
            # crossover 
            child = xover(parent1, parent2)
            
            # mutation           
            child = mutation(child)
            
            child.compute_fitness()
            new_population.append(child)

        population = new_population
   
    best_individual = max(population, key=lambda ind: ind.fitness)
    return best_individual

In [136]:
# Run  

for country in countries:

    init(country)
    print(country)
    best_solution = genetic_alg(country)

    best_solution.genome.append(best_solution.genome[0])  # Return to starting city
    tot_distance = compute_tour_distance(best_solution.genome, DIST_MATRIX)
    print(f"Genetic Algorithm with uniform selection, country: {country}")
    print(f"Result:  {len(best_solution.genome) - 1} steps, total length {tot_distance:.2f}km\n")




Russia


100%|██████████| 1000/1000 [00:20<00:00, 48.18it/s]


Genetic Algorithm with uniform selection, country: Russia
Result:  167 steps, total length 308726.43km

Italy


100%|██████████| 1000/1000 [00:02<00:00, 353.72it/s]


Genetic Algorithm with uniform selection, country: Italy
Result:  46 steps, total length 16832.64km

Vanuatu


...

Genetic Algorithm with uniform selection, country: Vanuatu
Result:  8 steps, total length 1397.64km

China


100%|██████████| 1000/1000 [05:38<00:00,  2.95it/s]


Genetic Algorithm with uniform selection, country: China
Result:  726 steps, total length 944914.05km

US


100%|██████████| 1000/1000 [01:08<00:00, 14.52it/s]

Genetic Algorithm with uniform selection, country: US
Result:  326 steps, total length 592699.61km



### Tournament selection

In [137]:
POPULATION_SIZE = 100
MAX_GENERATIONS = 1000
def genetic_alg( country='Italy'):
    
    population = []
    for _ in range(POPULATION_SIZE):
        unique_cities = random.sample(range(1, len(CITIES)), len(CITIES) - 1)
        genome = [0] + unique_cities 
        population.append(Individual(genome=genome))

    for ind in population:
        ind.compute_fitness()

    for _ in tqdm(range(MAX_GENERATIONS)):
        new_population = []

        while len(new_population) < POPULATION_SIZE:

            #tournament sel
            parent1 = parent_sel_tournament(population, 3)  #tournament size = 3
            parent2 = parent_sel_tournament(population, 3)
            
            # crossover 
            child = xover(parent1, parent2)
            
            # mutation           
            child = mutation(child)
            
            child.compute_fitness()
            new_population.append(child)

        population = new_population
   
    best_individual = max(population, key=lambda ind: ind.fitness)
    return best_individual

In [138]:
# Run  

for country in countries:

    init(country)
    print(country)
    best_solution = genetic_alg(country)

    best_solution.genome.append(best_solution.genome[0])  # Return to starting city
    tot_distance = compute_tour_distance(best_solution.genome, DIST_MATRIX)
    print(f"Genetic Algorithm with tournament seelction with size = 3, country: {country}")
    print(f"Result:  {len(best_solution.genome) - 1} steps, total length {tot_distance:.2f}km\n")

Russia


100%|██████████| 1000/1000 [00:20<00:00, 48.96it/s]


Genetic Algorithm with tournament seelction with size = 3, country: Russia
Result:  167 steps, total length 50564.93km

Italy


100%|██████████| 1000/1000 [00:02<00:00, 337.24it/s]


Genetic Algorithm with tournament seelction with size = 3, country: Italy
Result:  46 steps, total length 4259.66km

Vanuatu


...

Genetic Algorithm with tournament seelction with size = 3, country: Vanuatu
Result:  8 steps, total length 1345.54km

China


100%|██████████| 1000/1000 [05:15<00:00,  3.16it/s]


Genetic Algorithm with tournament seelction with size = 3, country: China
Result:  726 steps, total length 337189.47km

US


100%|██████████| 1000/1000 [01:08<00:00, 14.57it/s]

Genetic Algorithm with tournament seelction with size = 3, country: US
Result:  326 steps, total length 138031.48km

